In [ ]:
!echo -n "OS: "; cat /etc/os-release | grep PRETTY_NAME | cut -d= -f2 | tr -d '"'
!echo -n "GCC Version: "; gcc --version | head -n 1
!echo -n "CPU Model: "; lscpu | grep "Model name" | awk -F: '{print $2}' | xargs
!echo -n "Logical Cores: "; nproc
!echo -n "RAM: "; free -h --si | awk '/Mem:/ {print $2}'

In [ ]:
%pip install matplotlib

In [ ]:
import subprocess
import re
import matplotlib.pyplot as plt
import numpy as np
import os
import filecmp

In [ ]:
subprocess.run(['make'], check=True)

In [ ]:
def run_experiment(matrix_size, num_threads, seed=42, min_value=-1.0, max_value=1.0, data_dir = 'data'):
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    B_filename = os.path.join(data_dir, f'B_{matrix_size}.txt')
    C_filename = os.path.join(data_dir, f'C_{matrix_size}.txt')
    A_filename = os.path.join(data_dir, f'A_{matrix_size}_{num_threads}.txt')
    
    subprocess.run(['./save_random_matrix', B_filename, str(matrix_size), str(min_value), str(max_value), str(seed)], check=True)
    subprocess.run(['./save_random_matrix', C_filename, str(matrix_size), str(min_value), str(max_value), str(seed+1)], check=True)
    
    result = subprocess.run(['./main', B_filename, C_filename, A_filename, str(num_threads)], stdout=subprocess.PIPE, text=True)
    output = result.stdout
    
    match = re.search(r'Computation time: ([\d\.]+) seconds', output)
    if match:
        computation_time = float(match.group(1))
    else:
        raise RuntimeError("Failed to extract computation time")
    return computation_time, A_filename

In [ ]:
# B = {{1,2},{3,4}}
# C = {{5,6},{7,8}}
# M(C) = 6.5
# D(B) = 1.25
# A = B*C^2 + M(C)*I + I + D(B)*E = {{257.75, 291.25}, {566.25, 666.75}}

subprocess.run(['./main', 'data/B_sample.txt', 'data/C_sample.txt', 'data/A_sample_actual.txt', '1'], check=True)
if (filecmp.cmp("data/A_sample_actual.txt", "data/A_sample_expected.txt")):
    print("Seqential version results match manually calculated results")
else:
    print("Seqential version results and manually calculated results diverged!")

Computation time: 0.000015 seconds
Seqential version results match manually calculated results


In [ ]:
_, seq_output = run_experiment(matrix_size=100, num_threads=1)
_, par_output = run_experiment(matrix_size=100, num_threads=64)
if (filecmp.cmp(seq_output, par_output)):
    print("Parallel version results match sequential version results")
else:
    print("Parallel and sequential results diverged!")

In [ ]:
sizes = list(range(100, 3000, 200))
num_threads = 4

times_for_sizes = []
for size in sizes:
    time, _ = run_experiment(size, num_threads)
    times_for_sizes.append(time)
    print(f"Matrix size: {size}, Time: {time:.6f} seconds")

In [ ]:
plt.figure(figsize=(10,6))

plt.plot(sizes, times_for_sizes, marker='o')
plt.title('Computation Time for N-by-N Matrices')
plt.xlabel('N')
plt.ylabel('Computation Time (seconds)')
plt.grid(True)

plt.show()

In [ ]:
slope, _ = np.polyfit(np.log(sizes), np.log(times_for_sizes), deg=1)
float(slope)

In [ ]:
matrix_size = 2000
max_threads = 32

threads_list = list(range(1, max_threads+1))
times_for_threads = []

for num_threads in threads_list:
    time, _ = run_experiment(matrix_size, num_threads)
    times_for_threads.append(time)
    print(f'Number of threads: {num_threads}, Computation time: {time:.6f} seconds')

In [ ]:
base_time = times_for_threads[0]
speedups = [base_time / t for t in times_for_threads]

plt.figure(figsize=(10,6))
plt.plot(threads_list, speedups, marker='o')
plt.title('Speedup vs Number of Threads')
plt.xlabel('Number of Threads')
plt.ylabel('Speedup')
plt.grid(True)
plt.show()